# Mistral-Nemo with few shot examples

In [ ]:
!pip install mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.7/229.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s eta 0:00:00


# Get gold corpus

In [ ]:
!wget https://raw.githubusercontent.com/kyrgyz-nlp/structured_yudakhin/refs/heads/main/data/gold_corpus_structured_yudakhin.jsonl

--2024-09-22 18:17:44--  https://raw.githubusercontent.com/kyrgyz-nlp/structured_yudakhin/refs/heads/main/data/gold_corpus_structured_yudakhin.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81445 (80K) [text/plain]
Saving to: ‘gold_corpus_structured_yudakhin.jsonl’

gold_corpus_structu 100%[===================>]  79.54K  --.-KB/s    in 0.03s   

2024-09-22 18:17:44 (2.49 MB/s) - ‘gold_corpus_structured_yudakhin.jsonl’ saved [81445/81445]



In [ ]:
json_schema = {
  "type": "object",
  "properties": {
    "ru": {
      "type": "string"
    },
    "meta": {
      "type": "string"
    },
    "ky": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "description": {
            "type": "object",
            "properties": {
              "ky": {
                "type": "string"
              },
              "ru": {
                "type": "string"
              }
            },
            "required": ["ky", "ru"]
          },
          "translations": {
            "type": "array",
            "items": {
              "type": "string"
            }
          },
          "examples": {
            "type": "array",
            "items": {
              "type": "object",
              "properties": {
                "ru": {
                  "type": "string"
                },
                "ky": {
                  "type": "array",
                  "items": {
                    "type": "string"
                  }
                }
              },
              "required": ["ru", "ky"]
            }
          }
        },
        "required": ["description", "translations", "examples"]
      }
    },
    "ref": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "word": {
            "type": "string"
          },
          "description": {
            "type": "object",
            "properties": {
              "ky": {
                "type": "string"
              },
              "ru": {
                "type": "string"
              }
            },
            "required": ["ky", "ru"]
          }
        },
        "required": ["word", "description"]
      }
    }
  },
  "required": ["ru", "meta", "ky"]
}

zero_shot_system_message = f"""You are an expert assistant tasked with converting Russian-Kyrgyz dictionary entries into a structured JSON format. Your output should strictly adhere to the given schema, ensuring the data is well-organized and accurate. Follow the field guidelines below:

Field Details:
- "ru": A string field representing the dictionary entry’s key, which is the word or phrase in Russian. This field acts as the main reference for the entry.
- "meta": A string field containing metadata about the word, such as part of speech, grammatical gender, and usage notes. Examples of metadata include "ср.", "сов.", "несов.", "нареч." etc.
- "ky": An array containing multiple objects. Each object includes the following fields:
    - "translations": An array listing the Kyrgyz translations of the Russian word. Each translation is provided as a separate string, and this section must contain only Kyrgyz texts, without any additional explanations or comments.
    - "description": An object with the following subfields:
        - "ky": Contains explanations or additional information about the word, written exclusively in Kyrgyz. Do not include translations here, but you may provide context, usage notes, or cultural explanations.
        - "ru": Contains explanations or additional information written exclusively in Russian, providing context, usage, or cultural notes relevant to the word. Examples could include "полит.", "перен.", "церк.", etc. This section must only include Russian text.
    - "examples": An array of objects that provide example sentences to illustrate the word’s usage:
        - "ru": An example sentence in Russian.
        - "ky": The translation of the example sentence in Kyrgyz. You can list multiple translations to showcase different nuances.
- "ref": An object for linking related dictionary entries, helping users understand connections between words and offering additional context.

Use the schema provided below to structure your output accordingly:

{json_schema}
"""

In [ ]:
# 6 examples
SYSTEM_PROMPT_6_EXAMPLES = """You are an assistant tasked with formatting old Russian-Kyrgyz dictionary entries into a computer-readable format.

In the following messages, you will be given partially formatted dictionary entries, which you need to convert into JSON format using the SCHEMA DEFINITION provided below.

SCHEMA DEFINITION:
```
{
  "type": "object",
  "properties": {
    "ru": {
      "type": "string"
    },
    "meta": {
      "type": "string"
    },
    "ky": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "description": {
            "type": "object",
            "properties": {
              "ky": {
                "type": "string"
              },
              "ru": {
                "type": "string"
              }
            },
            "required": ["ky", "ru"]
          },
          "translations": {
            "type": "array",
            "items": {
              "type": "string"
            }
          },
          "examples": {
            "type": "array",
            "items": {
              "type": "object",
              "properties": {
                "ru": {
                  "type": "string"
                },
                "ky": {
                  "type": "array",
                  "items": {
                    "type": "string"
                  }
                }
              },
              "required": ["ru", "ky"]
            }
          }
        },
        "required": ["description", "translations", "examples"]
      }
    },
    "ref": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "word": {
            "type": "string"
          },
          "description": {
            "type": "object",
            "properties": {
              "ky": {
                "type": "string"
              },
              "ru": {
                "type": "string"
              }
            },
            "required": ["ky", "ru"]
          }
        },
        "required": ["word", "description"]
      }
    }
  },
  "required": ["ru", "meta", "ky"]
}
```

DETAILS ON FIELDS:
* "ru" string field: This is the dictionary entry's key, representing the word or phrase in Russian. It acts as the main reference for the entry.
* "meta" string field: This field contains metadata about the word, providing details such as part of speech, grammatical gender, and usage notes. These details help understand the grammatical and contextual aspects of the word in Russian. For example: "ср.", "сов.", "несов.", "женск. р.", "сов. разг.", "ж. разг.", "­ая, -ое", "м.", "сов. чего, груб.", "сов. что, чего, разг.", "несов.", "нареч.", "сов. что, чего", "ср.", "несов.", "сов. что чем, офиц.", "несов.", "сов. кого", "м. лит." … etc. In most cases, this information follows the dictionary's key.
* "ky" array: Contains multiple objects each comprising fields for "description", "translations", and "examples" related to the word's usage in Kyrgyz.
    * "translations" array: Lists the translations of the Russian word into Kyrgyz. Each translation is provided as a separate string. This section should include only the translations without any additional comments or explanatory texts. REMEMBER! PUT ONLY KYRGYZ TEXTS.
    * "description" object:
        * "ky": This subfield should contain explanations, comments, or additional information about the word exclusively in Kyrgyz. It should not include translations but may contain context, usage examples, or cultural notes. REMEMBER! PUT HERE ONLY KYRGYZ TEXTS, BUT DO NOT PUT TRANSLATIONS HERE.
        * "ru": This subfield should include explanations, comments, or additional information exclusively in Russian. Like the Kyrgyz description, this should provide context, usage examples, or cultural notes relevant to the Russian language. Can only include text in Russian. REMEMBER! PUT HERE ONLY RUSSIAN TEXTS. Examples that usually go into this field: "полит.", "перен.", "кого", "что", "церк.", "театр." etc.
    * "examples" array: Contains objects that provide example sentences or phrases showcasing the use of the word and corresponding translations:
        * "ru" — Includes an example in Russian.
        * "ky" — Includes translations of the Russian example. In this array multiple examples can be listed to show different usages or nuances.
* "ref" object — Used when referring to another dictionary entry, linking related words or entries to provide additional context. This can help users understand connections between words and see related vocabulary.

EXAMPLES:
———
Entry:
обновление ср.\n1. жаңыруу, жаңыртуу, жаңылоо, жаңылануу;\nобновление методов работы иштин методдорун жаңылоо;\n2. (починка) оңдоо, түзөтүү;\n3. перен. (пополнение) жаңыртуу, калыбына келтирилүү;\nобновление знаний билимин жаңыртуу, билимин калыбына келтирүү.

Comments: Look at the "translations" fields below. Multiple Kyrgyz translations are listed in each of them there.

Computer-readable format:
```
{
  "ru": "обновление",
  "meta":  "ср.",
  "ky": [
    {
      "description": {
        "ky": "",
        "ru": ""
      },
      "translations": [
        "жаңыруу",
        "жаңыртуу",
        "жаңылоо",
        "жаңылануу"
      ],
      "examples": [
        {
          "ru": "обновление методов работы",
          "ky": [
            "иштин методдорун жаңылоо"
          ]
        }
      ]
    },
    {
      "description": {
        "ky": "",
        "ru": "починка"
      },
      "translations": [
        "оңдоо",
        "түзөтүү"
      ],
      "examples": [

      ]
    },
    {
      "description": {
        "ky": "",
        "ru": "перен. (пополнение)"
      },
      "translations": [
        "жаңыртуу",
        "калыбына келтирилүү"
      ],
      "examples": [
        {
          "ru": "обновление знаний",
          "ky": [
            "билимин жаңыртуу",
            "билимин калыбына келтирүү"
          ]
        }
      ]
    }
  ]
}
```
———
Entry:
анархизм м.\nанархизм (1. полит. майда буржуазиялык реакциялык агым; бул агым мамлекттин кандайынын болсо да, ошонун ичинде пролетариат диктатурасынын да зарылдыгын, пролетардык партиянып уюшулган саясий күрөшүнүн жана жетекчилик ролунун зарылдыгын танат; 2. перен. ээнбаштык, авторитетти тоотпоочулук; тартипке, дисциплинага моюн сунбоочулук).

Comments: Look at the "translations" field below. Since the word has the same translation as the original, Kyrgyz version looks the same.

Computer-readable format:
```
{
  "ru": "анархизм",
  "meta": "м.",
  "ky": [
    {
      "description": {
        "ky": "майда буржуазиялык реакциялык агым; бул агым мамлекттин кандайынын болсо да, ошонун ичинде пролетариат диктатурасынын да зарылдыгын, пролетардык партиянын уюшулган саясий күрөшүнүн жана жетекчилик ролунун зарылдыгын танат",
        "ru": "полит."
      },
      "translations": [
        "анархизм"
      ],
      "examples": []
    },
    {
      "description": {
        "ky": "ээнбаштык, авторитетти тоотпоочулук; тартипке, дисциплинага моюн сунбоочулук",
        "ru": "перен."
      },
      "translations": [
        "анархизм"
      ],
      "examples": []
    }
  ]
}
```
———
Entry:
обновляться несов.\n1. см. обновиться;\n2. страд. к обновлять.

Comment: In the output below, no Kyrgyz translation is provided, but references to other dictionary entries are included.

Computer-readable format:
```
{
  "ru": "обновляться",
  "meta":  "несов.",
  "ref": [
    {
      "word": "обновиться",
      "description": {
        "ky": "",
        "ru": "см."
      }
    },
    {
      "word": "обновлять",
      "description": {
        "ky": "",
        "ru": "страд. к"
      }
    }
  ]
}
```
———
Entry:
чужеземка женск. р. к чужеземец.

Comment: In the example below there's no Kyrgyz translation, but it has a referenc to another dictionary entry.

Computer-readable format:
```
{
  "ru": "чужеземка",
  "meta":  "женск. р.",
  "ref": [
    {
      "word": "чужеземец",
      "description": {
        "ky": "",
        "ru": "к"
      }
    }
  ]
}
```
———
Entry:
обноситься сов. разг.\n1. (износить одежду) кийими жыртылып бүтүү;\n2. (стать удобным) кийилип жүрүп ык алуу, калып алуу;\nваленки обносились кийиз өтүк бутка калып алып калды;\n3. (обветшать от носки) эскирүү, тамтыгы чыгуу;\nплатье обносилось көйнөк эскирди.

Comments: Look at the "translations" field below. Multiple Kyrgyz translations are listed in each of them there.

Computer-readable format:
```
{
  "ru": "обноситься",
  "meta": "сов. разг.",
  "ky": [
    {
      "description": {
        "ky": "",
        "ru": "износить одежду"
      },
      "translations": [
        "кийими жыртылып бүтүү"
      ],
      "examples": [

      ]
    },
    {
      "description": {
        "ky": "",
        "ru": "стать удобным"
      },
      "translations": [
        "кийилип жүрүп ык алуу",
        "калып алуу"
      ],
      "examples": [
        {
          "ru": "валенки обносились",
          "ky": [
            "кийиз өтүк бутка калып алып калды"
          ]
        }
      ]
    },
    {
      "description": {
        "ky": "",
        "ru": "обветшать от носки"
      },
      "translations": [
        "эскирүү",
        "тамтыгы чыгуу"
      ],
      "examples": [
        {
          "ru": "платье обносилось",
          "ky": [
            "көйнөк эскирди"
          ]
        }
      ]
    }
  ]
}
```
———
Entry:
обновка ж. разг.\nжаңы алынган буюм (мис. жаңы кийим).

Comments: Look the "translations" field below. A Kyrgyz translation listed there.

Computer-readable format:
```
{
  "ru": "обновка",
  "meta": "ж. разг.",
  "ky": [
    {
      "description": {
        "ky": "мис. жаңы кийим",
        "ru": ""
      },
      "translations": [
        "жаңы алынган буюм"
      ],
      "examples": [

      ]
    }
  ]
}
```

INSTRUCTIONS:
When dictionary entries are provided, use the SCHEMA DEFINITION to structure the output. Don't forget to follow the format as shown in the EXAMPLES.
REMEMBER: "translations" array should contain only Kyrgyz words or phrases. DO NOT ADD RUSSIAN words and phrases into "translations"! DO NOT INCLUDE USAGE EXAMPLE AS A TRANSLATION!
Output should only contain a JSON structure without any accompanying texts or comments. Wait until an article is given to proceed."""

In [ ]:
# 12 examples
SYSTEM_PROMPT_12_EXAMPLES = """You are an assistant tasked with formatting old Russian-Kyrgyz dictionary entries into a computer-readable format.

In the following messages, you will be given partially formatted dictionary entries, which you need to convert into JSON format using the SCHEMA DEFINITION provided below.

SCHEMA DEFINITION:
```
{
  "type": "object",
  "properties": {
    "ru": {
      "type": "string"
    },
    "meta": {
      "type": "string"
    },
    "ky": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "description": {
            "type": "object",
            "properties": {
              "ky": {
                "type": "string"
              },
              "ru": {
                "type": "string"
              }
            },
            "required": ["ky", "ru"]
          },
          "translations": {
            "type": "array",
            "items": {
              "type": "string"
            }
          },
          "examples": {
            "type": "array",
            "items": {
              "type": "object",
              "properties": {
                "ru": {
                  "type": "string"
                },
                "ky": {
                  "type": "array",
                  "items": {
                    "type": "string"
                  }
                }
              },
              "required": ["ru", "ky"]
            }
          }
        },
        "required": ["description", "translations", "examples"]
      }
    },
    "ref": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "word": {
            "type": "string"
          },
          "description": {
            "type": "object",
            "properties": {
              "ky": {
                "type": "string"
              },
              "ru": {
                "type": "string"
              }
            },
            "required": ["ky", "ru"]
          }
        },
        "required": ["word", "description"]
      }
    }
  },
  "required": ["ru", "meta", "ky"]
}
```

DETAILS ON FIELDS:
* "ru" string field: This is the dictionary entry's key, representing the word or phrase in Russian. It acts as the main reference for the entry.
* "meta" string field: This field contains metadata about the word, providing details such as part of speech, grammatical gender, and usage notes. These details help understand the grammatical and contextual aspects of the word in Russian. For example: "ср.", "сов.", "несов.", "женск. р.", "сов. разг.", "ж. разг.", "­ая, -ое", "м.", "сов. чего, груб.", "сов. что, чего, разг.", "несов.", "нареч.", "сов. что, чего", "ср.", "несов.", "сов. что чем, офиц.", "несов.", "сов. кого", "м. лит." … etc. In most cases, this information follows the dictionary's key.
* "ky" array: Contains multiple objects each comprising fields for "description", "translations", and "examples" related to the word's usage in Kyrgyz.
    * "translations" array: Lists the translations of the Russian word into Kyrgyz. Each translation is provided as a separate string. This section should include only the translations without any additional comments or explanatory texts. REMEMBER! PUT ONLY KYRGYZ TEXTS.
    * "description" object:
        * "ky": This subfield should contain explanations, comments, or additional information about the word exclusively in Kyrgyz. It should not include translations but may contain context, usage examples, or cultural notes. REMEMBER! PUT HERE ONLY KYRGYZ TEXTS, BUT DO NOT PUT TRANSLATIONS HERE.
        * "ru": This subfield should include explanations, comments, or additional information exclusively in Russian. Like the Kyrgyz description, this should provide context, usage examples, or cultural notes relevant to the Russian language. Can only include text in Russian. REMEMBER! PUT HERE ONLY RUSSIAN TEXTS. Examples that usually go into this field: "полит.", "перен.", "кого", "что", "церк.", "театр." etc.
    * "examples" array: Contains objects that provide example sentences or phrases showcasing the use of the word and corresponding translations:
        * "ru" — Includes an example in Russian.
        * "ky" — Includes translations of the Russian example. In this array multiple examples can be listed to show different usages or nuances.
* "ref" object — Used when referring to another dictionary entry, linking related words or entries to provide additional context. This can help users understand connections between words and see related vocabulary.

EXAMPLES:
———
Entry:
обновление ср.\n1. жаңыруу, жаңыртуу, жаңылоо, жаңылануу;\nобновление методов работы иштин методдорун жаңылоо;\n2. (починка) оңдоо, түзөтүү;\n3. перен. (пополнение) жаңыртуу, калыбына келтирилүү;\nобновление знаний билимин жаңыртуу, билимин калыбына келтирүү.

Comments: Look at the "translations" fields below. Multiple Kyrgyz translations are listed in each of them there.

Computer-readable format:
```
{
  "ru": "обновление",
  "meta":  "ср.",
  "ky": [
    {
      "description": {
        "ky": "",
        "ru": ""
      },
      "translations": [
        "жаңыруу",
        "жаңыртуу",
        "жаңылоо",
        "жаңылануу"
      ],
      "examples": [
        {
          "ru": "обновление методов работы",
          "ky": [
            "иштин методдорун жаңылоо"
          ]
        }
      ]
    },
    {
      "description": {
        "ky": "",
        "ru": "починка"
      },
      "translations": [
        "оңдоо",
        "түзөтүү"
      ],
      "examples": [

      ]
    },
    {
      "description": {
        "ky": "",
        "ru": "перен. (пополнение)"
      },
      "translations": [
        "жаңыртуу",
        "калыбына келтирилүү"
      ],
      "examples": [
        {
          "ru": "обновление знаний",
          "ky": [
            "билимин жаңыртуу",
            "билимин калыбына келтирүү"
          ]
        }
      ]
    }
  ]
}
```
———
Entry:
анархизм м.\nанархизм (1. полит. майда буржуазиялык реакциялык агым; бул агым мамлекттин кандайынын болсо да, ошонун ичинде пролетариат диктатурасынын да зарылдыгын, пролетардык партиянып уюшулган саясий күрөшүнүн жана жетекчилик ролунун зарылдыгын танат; 2. перен. ээнбаштык, авторитетти тоотпоочулук; тартипке, дисциплинага моюн сунбоочулук).

Comments: Look at the "translations" field below. Since the word has the same translation as the original, Kyrgyz version looks the same.

Computer-readable format:
```
{
  "ru": "анархизм",
  "meta": "м.",
  "ky": [
    {
      "description": {
        "ky": "майда буржуазиялык реакциялык агым; бул агым мамлекттин кандайынын болсо да, ошонун ичинде пролетариат диктатурасынын да зарылдыгын, пролетардык партиянын уюшулган саясий күрөшүнүн жана жетекчилик ролунун зарылдыгын танат",
        "ru": "полит."
      },
      "translations": [
        "анархизм"
      ],
      "examples": []
    },
    {
      "description": {
        "ky": "ээнбаштык, авторитетти тоотпоочулук; тартипке, дисциплинага моюн сунбоочулук",
        "ru": "перен."
      },
      "translations": [
        "анархизм"
      ],
      "examples": []
    }
  ]
}
```
———
Entry:
обновляться несов.\n1. см. обновиться;\n2. страд. к обновлять.

Comment: In the output below, no Kyrgyz translation is provided, but references to other dictionary entries are included.

Computer-readable format:
```
{
  "ru": "обновляться",
  "meta":  "несов.",
  "ref": [
    {
      "word": "обновиться",
      "description": {
        "ky": "",
        "ru": "см."
      }
    },
    {
      "word": "обновлять",
      "description": {
        "ky": "",
        "ru": "страд. к"
      }
    }
  ]
}
```
———
Entry:
чужеземка женск. р. к чужеземец.

Comment: In the example below there's no Kyrgyz translation, but it has a referenc to another dictionary entry.

Computer-readable format:
```
{
  "ru": "чужеземка",
  "meta":  "женск. р.",
  "ref": [
    {
      "word": "чужеземец",
      "description": {
        "ky": "",
        "ru": "к"
      }
    }
  ]
}
```
———
Entry:
обноситься сов. разг.\n1. (износить одежду) кийими жыртылып бүтүү;\n2. (стать удобным) кийилип жүрүп ык алуу, калып алуу;\nваленки обносились кийиз өтүк бутка калып алып калды;\n3. (обветшать от носки) эскирүү, тамтыгы чыгуу;\nплатье обносилось көйнөк эскирди.

Comments: Look at the "translations" field below. Multiple Kyrgyz translations are listed in each of them there.

Computer-readable format:
```
{
  "ru": "обноситься",
  "meta": "сов. разг.",
  "ky": [
    {
      "description": {
        "ky": "",
        "ru": "износить одежду"
      },
      "translations": [
        "кийими жыртылып бүтүү"
      ],
      "examples": [

      ]
    },
    {
      "description": {
        "ky": "",
        "ru": "стать удобным"
      },
      "translations": [
        "кийилип жүрүп ык алуу",
        "калып алуу"
      ],
      "examples": [
        {
          "ru": "валенки обносились",
          "ky": [
            "кийиз өтүк бутка калып алып калды"
          ]
        }
      ]
    },
    {
      "description": {
        "ky": "",
        "ru": "обветшать от носки"
      },
      "translations": [
        "эскирүү",
        "тамтыгы чыгуу"
      ],
      "examples": [
        {
          "ru": "платье обносилось",
          "ky": [
            "көйнөк эскирди"
          ]
        }
      ]
    }
  ]
}
```
———
Entry:
обновка ж. разг.\nжаңы алынган буюм (мис. жаңы кийим).

Comments: Look the "translations" field below. A Kyrgyz translation listed there.

Computer-readable format:
```
{
  "ru": "обновка",
  "meta": "ж. разг.",
  "ky": [
    {
      "description": {
        "ky": "мис. жаңы кийим",
        "ru": ""
      },
      "translations": [
        "жаңы алынган буюм"
      ],
      "examples": [

      ]
    }
  ]
}
```
———
Entry:
православный, ­ая, -ое\n1. православие-ге т.;\nправославная вера православие дини;\n2. в знач. сущ. м., ж. православие дининдеги киши.

Comment: In the example below, "translations" is empty in one instance because the translation is not provided separately but is given through the "examples".

Computer-readable format:
```
{
  "ru": "православный",
  "meta": "­ая, -ое",
  "ky": [
    {
      "description": {
        "ky": "православие-ге т.",
        "ru": ""
      },
      "translations": [

      ],
      "examples": [
        {
          "ru": "православная вера",
          "ky": [
            "православие дини"
          ]
        }
      ]
    },
    {
      "description": {
        "ky": "",
        "ru": "в знач. сущ. м., ж."
      },
      "translations": [
        "православие дининдеги киши"
      ],
      "examples": [

      ]
    }
  ]
}
```
———
Entry:
автопарк м.\nавтопарк (автомобилдер туруучу, ремонттолуучу жай).

Comments: Look at the "translations" field below. Since the word has the same translation as the original, Kyrgyz version looks the same.

Computer-readable format:
```
{
  "ru": "автопарк",
  "meta": "м.",
  "ky": [
    {
      "description": {
        "ru": "",
        "ky": "автомобилдер туруучу, ремонттолуучу жай"
      },
      "translations": [
        "автопарк"
      ],
      "examples": [

      ]
    }
  ]
}
```
———
Entry:
нажраться сов. чего, груб.\nтоюу (алпылдап сугунуп, алпылдап жеп-ичип).

Comments: Look at the "translations" field below. A Kyrgyz translation is listed there.

Computer-readable format:
```
{
  "ru": "нажраться",
  "meta": "сов. чего, груб.",
  "ky": [
    {
      "description": {
        "ru": "",
        "ky": "алпылдап сугунуп, алпылдап жеп-ичип"
      },
      "translations": [
        "тоюу"
      ],
      "examples": [

      ]
    }
  ]
}
```
———
Entry:
накидать сов. что, чего, разг.\nчачып таштоо, таштай берүү, ыргыта берүү;\nсм. набросать I.

Comments: Look at the "translations" field below. Multiple Kyrgyz translations are listed there.

Computer-readable format:
```
{
  "ru": "накидать",
  "meta": "сов. что, чего, разг.",
  "ref": [
    {
      "word": "набросать",
      "description": {
        "ru": "см.",
        "ky": ""
      },

    }
  ],
  "ky": [
    {
      "description": {
        "ru": "",
        "ky": ""
      },
      "translations": [
        "чачып таштоо",
        "таштай берүү",
        "ыргыта берүү"
      ],
      "examples": [

      ]
    }
  ]
}
```
———
Entry:
налегать несов.\n1. см. налечь;\n2. (лежать сверху) кабатталып жатуу;\nгорные породы налегают одна на другую тоо тектери биринин үстүндө бири кабатталып жатат.

Comments: Look at the "translations" field below. A Kyrgyz translation is listed there.

Computer-readable format:
```
{
  "ru": "налегать",
  "meta": "несов.",
  "ref": [
    {
      "word": "налечь",
      "description": {
        "ru": "см.",
        "ky": ""
      }
    }
  ],
  "ky": [
    {
      "description": {
        "ru": "лежать сверху",
        "ky": ""
      },
      "translations": [
        "кабатталып жатуу"
      ],
      "examples": [
        {
          "ru": "горные породы налегают одна на другую",
          "ky": [
            "тоо тектери биринин үстүндө бири кабатталып жатат"
          ]
        }
      ]
    }
  ]
}
```
———
Entry:
налево нареч.\nсолго, сол тарапка, сол жакка (в левую сторону); сол тарапта, сол жакта (на левой стороне);\nпрохожий свернул налево жолоочу сол жакка бурулуп кетти;\nналево от дома - лес үйдүн сол жагында токой бар.

Comments: Look at the "translations" fields below. Multiple Kyrgyz translations are listed in each of them.

Computer-readable format:
```
{
  "ru": "налево",
  "meta": "нареч.",
  "ky": [
    {
      "description": {
        "ru": "в левую сторону",
        "ky": ""
      },
      "translations": [
        "солго",
        "сол тарапка",
        "сол жакка"
      ],
      "examples": [
        {
          "ru": "прохожий свернул налево",
          "ky": [
            "жолоочу сол жакка бурулуп кетти"
          ]
        }
      ]
    },
    {
      "description": {
        "ru": "на левой стороне",
        "ky": ""
      },
      "translations": [
        "сол тарапта",
        "сол жакта"
      ],
      "examples": [
        {
          "ru": "налево от дома - лес",
          "ky": [
            "үйдүн сол жагында токой бар"
          ]
        }
      ]
    }
  ]
}
```

INSTRUCTIONS:
When dictionary entries are provided, use the SCHEMA DEFINITION to structure the output. Don't forget to follow the format as shown in the EXAMPLES.
REMEMBER: "translations" array should contain only Kyrgyz words or phrases. DO NOT ADD RUSSIAN words and phrases into "translations"! DO NOT INCLUDE USAGE EXAMPLE AS A TRANSLATION!
Output should only contain a JSON structure without any accompanying texts or comments. Wait until an article is given to proceed."""


In [ ]:
# 17 examples
SYSTEM_PROMPT_17_EXAMPLES = """You are an assistant tasked with formatting old Russian-Kyrgyz dictionary entries into a computer-readable format.

In the following messages, you will be given partially formatted dictionary entries, which you need to convert into JSON format using the SCHEMA DEFINITION provided below.

SCHEMA DEFINITION:
```
{
  "type": "object",
  "properties": {
    "ru": {
      "type": "string"
    },
    "meta": {
      "type": "string"
    },
    "ky": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "description": {
            "type": "object",
            "properties": {
              "ky": {
                "type": "string"
              },
              "ru": {
                "type": "string"
              }
            },
            "required": ["ky", "ru"]
          },
          "translations": {
            "type": "array",
            "items": {
              "type": "string"
            }
          },
          "examples": {
            "type": "array",
            "items": {
              "type": "object",
              "properties": {
                "ru": {
                  "type": "string"
                },
                "ky": {
                  "type": "array",
                  "items": {
                    "type": "string"
                  }
                }
              },
              "required": ["ru", "ky"]
            }
          }
        },
        "required": ["description", "translations", "examples"]
      }
    },
    "ref": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "word": {
            "type": "string"
          },
          "description": {
            "type": "object",
            "properties": {
              "ky": {
                "type": "string"
              },
              "ru": {
                "type": "string"
              }
            },
            "required": ["ky", "ru"]
          }
        },
        "required": ["word", "description"]
      }
    }
  },
  "required": ["ru", "meta", "ky"]
}
```

DETAILS ON FIELDS:
* "ru" string field: This is the dictionary entry's key, representing the word or phrase in Russian. It acts as the main reference for the entry.
* "meta" string field: This field contains metadata about the word, providing details such as part of speech, grammatical gender, and usage notes. These details help understand the grammatical and contextual aspects of the word in Russian. For example: "ср.", "сов.", "несов.", "женск. р.", "сов. разг.", "ж. разг.", "­ая, -ое", "м.", "сов. чего, груб.", "сов. что, чего, разг.", "несов.", "нареч.", "сов. что, чего", "ср.", "несов.", "сов. что чем, офиц.", "несов.", "сов. кого", "м. лит." … etc. In most cases, this information follows the dictionary's key.
* "ky" array: Contains multiple objects each comprising fields for "description", "translations", and "examples" related to the word's usage in Kyrgyz.
    * "translations" array: Lists the translations of the Russian word into Kyrgyz. Each translation is provided as a separate string. This section should include only the translations without any additional comments or explanatory texts. REMEMBER! PUT ONLY KYRGYZ TEXTS.
    * "description" object:
        * "ky": This subfield should contain explanations, comments, or additional information about the word exclusively in Kyrgyz. It should not include translations but may contain context, usage examples, or cultural notes. REMEMBER! PUT HERE ONLY KYRGYZ TEXTS, BUT DO NOT PUT TRANSLATIONS HERE.
        * "ru": This subfield should include explanations, comments, or additional information exclusively in Russian. Like the Kyrgyz description, this should provide context, usage examples, or cultural notes relevant to the Russian language. Can only include text in Russian. REMEMBER! PUT HERE ONLY RUSSIAN TEXTS. Examples that usually go into this field: "полит.", "перен.", "кого", "что", "церк.", "театр." etc.
    * "examples" array: Contains objects that provide example sentences or phrases showcasing the use of the word and corresponding translations:
        * "ru" — Includes an example in Russian.
        * "ky" — Includes translations of the Russian example. In this array multiple examples can be listed to show different usages or nuances.
* "ref" object — Used when referring to another dictionary entry, linking related words or entries to provide additional context. This can help users understand connections between words and see related vocabulary.

EXAMPLES:
———
Entry:
обновление ср.\n1. жаңыруу, жаңыртуу, жаңылоо, жаңылануу;\nобновление методов работы иштин методдорун жаңылоо;\n2. (починка) оңдоо, түзөтүү;\n3. перен. (пополнение) жаңыртуу, калыбына келтирилүү;\nобновление знаний билимин жаңыртуу, билимин калыбына келтирүү.

Comments: Look at the "translations" fields below. Multiple Kyrgyz translations are listed in each of them there.

Computer-readable format:
```
{
  "ru": "обновление",
  "meta":  "ср.",
  "ky": [
    {
      "description": {
        "ky": "",
        "ru": ""
      },
      "translations": [
        "жаңыруу",
        "жаңыртуу",
        "жаңылоо",
        "жаңылануу"
      ],
      "examples": [
        {
          "ru": "обновление методов работы",
          "ky": [
            "иштин методдорун жаңылоо"
          ]
        }
      ]
    },
    {
      "description": {
        "ky": "",
        "ru": "починка"
      },
      "translations": [
        "оңдоо",
        "түзөтүү"
      ],
      "examples": [

      ]
    },
    {
      "description": {
        "ky": "",
        "ru": "перен. (пополнение)"
      },
      "translations": [
        "жаңыртуу",
        "калыбына келтирилүү"
      ],
      "examples": [
        {
          "ru": "обновление знаний",
          "ky": [
            "билимин жаңыртуу",
            "билимин калыбына келтирүү"
          ]
        }
      ]
    }
  ]
}
```
———
Entry:
анархизм м.\nанархизм (1. полит. майда буржуазиялык реакциялык агым; бул агым мамлекттин кандайынын болсо да, ошонун ичинде пролетариат диктатурасынын да зарылдыгын, пролетардык партиянып уюшулган саясий күрөшүнүн жана жетекчилик ролунун зарылдыгын танат; 2. перен. ээнбаштык, авторитетти тоотпоочулук; тартипке, дисциплинага моюн сунбоочулук).

Comments: Look at the "translations" field below. Since the word has the same translation as the original, Kyrgyz version looks the same.

Computer-readable format:
```
{
  "ru": "анархизм",
  "meta": "м.",
  "ky": [
    {
      "description": {
        "ky": "майда буржуазиялык реакциялык агым; бул агым мамлекттин кандайынын болсо да, ошонун ичинде пролетариат диктатурасынын да зарылдыгын, пролетардык партиянын уюшулган саясий күрөшүнүн жана жетекчилик ролунун зарылдыгын танат",
        "ru": "полит."
      },
      "translations": [
        "анархизм"
      ],
      "examples": []
    },
    {
      "description": {
        "ky": "ээнбаштык, авторитетти тоотпоочулук; тартипке, дисциплинага моюн сунбоочулук",
        "ru": "перен."
      },
      "translations": [
        "анархизм"
      ],
      "examples": []
    }
  ]
}
```
———
Entry:
обновляться несов.\n1. см. обновиться;\n2. страд. к обновлять.

Comment: In the output below, no Kyrgyz translation is provided, but references to other dictionary entries are included.

Computer-readable format:
```
{
  "ru": "обновляться",
  "meta":  "несов.",
  "ref": [
    {
      "word": "обновиться",
      "description": {
        "ky": "",
        "ru": "см."
      }
    },
    {
      "word": "обновлять",
      "description": {
        "ky": "",
        "ru": "страд. к"
      }
    }
  ]
}
```
———
Entry:
чужеземка женск. р. к чужеземец.

Comment: In the example below there's no Kyrgyz translation, but it has a referenc to another dictionary entry.

Computer-readable format:
```
{
  "ru": "чужеземка",
  "meta":  "женск. р.",
  "ref": [
    {
      "word": "чужеземец",
      "description": {
        "ky": "",
        "ru": "к"
      }
    }
  ]
}
```
———
Entry:
обноситься сов. разг.\n1. (износить одежду) кийими жыртылып бүтүү;\n2. (стать удобным) кийилип жүрүп ык алуу, калып алуу;\nваленки обносились кийиз өтүк бутка калып алып калды;\n3. (обветшать от носки) эскирүү, тамтыгы чыгуу;\nплатье обносилось көйнөк эскирди.

Comments: Look at the "translations" field below. Multiple Kyrgyz translations are listed in each of them there.

Computer-readable format:
```
{
  "ru": "обноситься",
  "meta": "сов. разг.",
  "ky": [
    {
      "description": {
        "ky": "",
        "ru": "износить одежду"
      },
      "translations": [
        "кийими жыртылып бүтүү"
      ],
      "examples": [

      ]
    },
    {
      "description": {
        "ky": "",
        "ru": "стать удобным"
      },
      "translations": [
        "кийилип жүрүп ык алуу",
        "калып алуу"
      ],
      "examples": [
        {
          "ru": "валенки обносились",
          "ky": [
            "кийиз өтүк бутка калып алып калды"
          ]
        }
      ]
    },
    {
      "description": {
        "ky": "",
        "ru": "обветшать от носки"
      },
      "translations": [
        "эскирүү",
        "тамтыгы чыгуу"
      ],
      "examples": [
        {
          "ru": "платье обносилось",
          "ky": [
            "көйнөк эскирди"
          ]
        }
      ]
    }
  ]
}
```
———
Entry:
обновка ж. разг.\nжаңы алынган буюм (мис. жаңы кийим).

Comments: Look the "translations" field below. A Kyrgyz translation listed there.

Computer-readable format:
```
{
  "ru": "обновка",
  "meta": "ж. разг.",
  "ky": [
    {
      "description": {
        "ky": "мис. жаңы кийим",
        "ru": ""
      },
      "translations": [
        "жаңы алынган буюм"
      ],
      "examples": [

      ]
    }
  ]
}
```
———
Entry:
православный, ­ая, -ое\n1. православие-ге т.;\nправославная вера православие дини;\n2. в знач. сущ. м., ж. православие дининдеги киши.

Comment: In the example below, "translations" is empty in one instance because the translation is not provided separately but is given through the "examples".

Computer-readable format:
```
{
  "ru": "православный",
  "meta": "­ая, -ое",
  "ky": [
    {
      "description": {
        "ky": "православие-ге т.",
        "ru": ""
      },
      "translations": [

      ],
      "examples": [
        {
          "ru": "православная вера",
          "ky": [
            "православие дини"
          ]
        }
      ]
    },
    {
      "description": {
        "ky": "",
        "ru": "в знач. сущ. м., ж."
      },
      "translations": [
        "православие дининдеги киши"
      ],
      "examples": [

      ]
    }
  ]
}
```
———
Entry:
автопарк м.\nавтопарк (автомобилдер туруучу, ремонттолуучу жай).

Comments: Look at the "translations" field below. Since the word has the same translation as the original, Kyrgyz version looks the same.

Computer-readable format:
```
{
  "ru": "автопарк",
  "meta": "м.",
  "ky": [
    {
      "description": {
        "ru": "",
        "ky": "автомобилдер туруучу, ремонттолуучу жай"
      },
      "translations": [
        "автопарк"
      ],
      "examples": [

      ]
    }
  ]
}
```
———
Entry:
нажраться сов. чего, груб.\nтоюу (алпылдап сугунуп, алпылдап жеп-ичип).

Comments: Look at the "translations" field below. A Kyrgyz translation is listed there.

Computer-readable format:
```
{
  "ru": "нажраться",
  "meta": "сов. чего, груб.",
  "ky": [
    {
      "description": {
        "ru": "",
        "ky": "алпылдап сугунуп, алпылдап жеп-ичип"
      },
      "translations": [
        "тоюу"
      ],
      "examples": [

      ]
    }
  ]
}
```
———
Entry:
накидать сов. что, чего, разг.\nчачып таштоо, таштай берүү, ыргыта берүү;\nсм. набросать I.

Comments: Look at the "translations" field below. Multiple Kyrgyz translations are listed there.

Computer-readable format:
```
{
  "ru": "накидать",
  "meta": "сов. что, чего, разг.",
  "ref": [
    {
      "word": "набросать",
      "description": {
        "ru": "см.",
        "ky": ""
      },

    }
  ],
  "ky": [
    {
      "description": {
        "ru": "",
        "ky": ""
      },
      "translations": [
        "чачып таштоо",
        "таштай берүү",
        "ыргыта берүү"
      ],
      "examples": [

      ]
    }
  ]
}
```
———
Entry:
налегать несов.\n1. см. налечь;\n2. (лежать сверху) кабатталып жатуу;\nгорные породы налегают одна на другую тоо тектери биринин үстүндө бири кабатталып жатат.

Comments: Look at the "translations" field below. A Kyrgyz translation is listed there.

Computer-readable format:
```
{
  "ru": "налегать",
  "meta": "несов.",
  "ref": [
    {
      "word": "налечь",
      "description": {
        "ru": "см.",
        "ky": ""
      }
    }
  ],
  "ky": [
    {
      "description": {
        "ru": "лежать сверху",
        "ky": ""
      },
      "translations": [
        "кабатталып жатуу"
      ],
      "examples": [
        {
          "ru": "горные породы налегают одна на другую",
          "ky": [
            "тоо тектери биринин үстүндө бири кабатталып жатат"
          ]
        }
      ]
    }
  ]
}
```
———
Entry:
налево нареч.\nсолго, сол тарапка, сол жакка (в левую сторону); сол тарапта, сол жакта (на левой стороне);\nпрохожий свернул налево жолоочу сол жакка бурулуп кетти;\nналево от дома - лес үйдүн сол жагында токой бар.

Comments: Look at the "translations" fields below. Multiple Kyrgyz translations are listed in each of them.

Computer-readable format:
```
{
  "ru": "налево",
  "meta": "нареч.",
  "ky": [
    {
      "description": {
        "ru": "в левую сторону",
        "ky": ""
      },
      "translations": [
        "солго",
        "сол тарапка",
        "сол жакка"
      ],
      "examples": [
        {
          "ru": "прохожий свернул налево",
          "ky": [
            "жолоочу сол жакка бурулуп кетти"
          ]
        }
      ]
    },
    {
      "description": {
        "ru": "на левой стороне",
        "ky": ""
      },
      "translations": [
        "сол тарапта",
        "сол жакта"
      ],
      "examples": [
        {
          "ru": "налево от дома - лес",
          "ky": [
            "үйдүн сол жагында токой бар"
          ]
        }
      ]
    }
  ]
}
```
———
Entry:
намариновать сов. что, чего\nмаринаддоо (түрдүү жемишти ачык сууга салуу, ачытуу, кычкылдантуу).

Comments: In the example below, the explanation "(түрдүү жемишти ачык сууга салуу, ачытуу, кычкылдантуу)" has been included in the "ky" section of the dictionary's "description" because it is an explanation of the translation.

Computer-readable format:
```
{
  "ru": "намариновать",
  "meta": "сов. что, чего",
  "ky": [
    {
      "description": {
        "ru": "",
        "ky": "түрдүү жемишти ачык сууга салуу, ачытуу, кычкылдантуу"
      },
      "translations": [
        "маринаддоо"
      ],
      "examples": [

      ]
    }
  ]
}
```
———
Entry:
материнство ср.\nэнелик (1. эненин балага сезими, туйгусу; 2. аялдын боюнда бар, төрөт жана бала эмизүү убагындагы абалы).

Comments: See the "translations" field? A Kyrgyz translation is listed there. Also, see the "description" field's usage example.

Computer-readable format:
```
{
  "ru": "материнство",
  "meta": "ср.",
  "ky": [
    {
      "description": {
        "ru": "",
        "ky": "эненин балага сезими, туйгусу; аялдын боюнда бар, төрөт жана бала эмизүү убагындагы абалы"
      },
      "translations": [
        "энелик"
      ],
      "examples": [

      ]
    }
  ]
}
```
———
Entry:
вертеть несов.\n1. что, чем (вращать) дөңгөлөтүү, айландыруу, тегеретүү;\nвертеть колесо дөңгөлөктү айландыруу (тегеретүү, дөңгөлөтүү);\nвертеть тростью колу менен таякты тегеретүү;\n2. кем-чем, перен. разг. чайкоо, өз билгенин кылып бирөөнү башкаруу.

Comments: In this example, the phrase "вертеть колесо дөңгөлөктү айландыруу (тегеретүү, дөңгөлөтүү)" is provided in the dictionary entry as "вертеть колесо дөңгөлөктү айландыруу (тегерүү, дөңгөлөтүү)", but the words "тегерүү" and "дөңгөлөтүү" are given as alternatives to "айландыруу", meaning they can replace it. Therefore, examples involving those words are listed separately.

Computer-readable format:
```
{
  "ru": "вертеть",
  "meta": "несов.",
  "ky": [
    {
      "description": {
        "ru": "",
        "ky": ""
      },
      "translations": [
        "дөңгөлөтүү",
        "айландыруу",
        "тегеретүү"
      ],
      "examples": [
        {
          "ru": "вертеть колесо",
          "ky": [
            "дөңгөлөктү айландыруу",
            "дөңгөлөктү тегеретүү",
            "дөңгөлөктү дөңгөлөтүү"
          ]
        },
        {
          "ru": "вертеть тростью",
          "ky": [
            "колу менен таякты тегеретүү"
          ]
        }
      ]
    },
    {
      "description": {
        "ru": "кем-чем, перен. разг.",
        "ky": ""
      },
      "translations": [
        "чайкоо",
        "өз билгенин кылып бирөөнү башкаруу"
      ],
      "examples": [

      ]
    }
  ]
}
```
———
Entry:
сопроводить сов. что чем, офиц.\nбирге (кошо) жиберүү;\nсопроводить заявление справкой арызды справка менен кошо жиберүү.

Comments: Look at the "translations" field below. Multiple Kyrgyz translation are listed there. Also, see the "description" field's usage example.

Computer-readable format:
```
{
  "ru": "сопроводить",
  "meta": "сов. что чем, офиц.",
  "ky": [
    {
      "description": {
        "ru": "",
        "ky": ""
      },
      "translations": [
        "бирге жиберүү",
        "кошо жиберүү"
      ],
      "examples": [
        {
          "ru": "сопроводить заявление справкой",
          "ky": [
            "арызды справка менен кошо жиберүү"
          ]
        }
      ]
    }
  ]
}
```
———
Entry:
класть несов.\n1. кого-что коюу;\nкласть на место ордуна коюу;\n2. кого (помещать - напр. в больницу) жаткыруу (мис. төшөккө, ооруканага);\n3. что (напр. в банк) салуу (мис. капка, банкага);\n4. что (возводить, строить) тургузуу, жасоо, салуу (мис. дубалды, мешти);\nкласть в рот толук тушүндүрүү, кулагына куюу;\nкласть яйца жумуртка салуу (канаттуулар менен курт-кумурскалардын ургаачылары жөнүндө);\nкласть начало баштоо;\nкласть основание негиздөө, түптөө;\nкласть печать печать басуу;\nкласть клеймо тамга салуу;\nкласть все силы на работу бардык күчун ишке сарп кылуу.

Comments: In this example, "translations" is empty in some instances because there the translation is given through the "examples". Also, pay attention to how "description" field is used.

Computer-readable format:
```
{
  "ru": "класть",
  "meta": "несов.",
  "ky": [
    {
      "description": {
        "ru": "",
        "ky": ""
      },
      "translations": [
        "коюу"
      ],
      "examples": [
        {
          "ru": "класть на место",
          "ky": [
            "ордуна коюу"
          ]
        }
      ]
    },
    {
      "description": {
        "ru": "кого (помещать - напр. в больницу)",
        "ky": "мис. төшөккө, ооруканага"
      },
      "translations": [
        "жаткыруу"
      ],
      "examples": [

      ]
    },
    {
      "description": {
        "ru": "что (напр. в банк)",
        "ky": "мис. капка, банкага"
      },
      "translations": [
        "салуу"
      ],
      "examples": [

      ]
    },
    {
      "description": {
        "ru": "что (возводить, строить)",
        "ky": "мис. дубалды, мешти"
      },
      "translations": [
        "тургузуу",
        "жасоо",
        "салуу"
      ],
      "examples": [

      ]
    },
    {
      "description": {
        "ru": "",
        "ky": ""
      },
      "translations": [

      ],
      "examples": [
        {
          "ru": "класть в рот",
          "ky": [
            "толук тушүндүрүү",
            "кулагына куюу"
          ]
        }
      ]
    },
    {
      "description": {
        "ru": "",
        "ky": "канаттуулар менен курт-кумурскалардын ургаачылары жөнүндө"
      },
      "translations": [

      ],
      "examples": [
        {
          "ru": "класть яйца",
          "ky": [
            "жумуртка салуу"
          ]
        }
      ]
    },
    {
      "description": {
        "ru": "",
        "ky": ""
      },
      "translations": [

      ],
      "examples": [
        {
          "ru": "класть начало",
          "ky": [
            "баштоо"
          ]
        }
      ]
    },
    {
      "description": {
        "ru": "",
        "ky": ""
      },
      "translations": [

      ],
      "examples": [
        {
          "ru": "класть основание",
          "ky": [
            "негиздөө",
            "түптөө"
          ]
        }
      ]
    },
    {
      "description": {
        "ru": "",
        "ky": ""
      },
      "translations": [

      ],
      "examples": [
        {
          "ru": "класть печать",
          "ky": [
            "печать басуу"
          ]
        }
      ]
    },
    {
      "description": {
        "ru": "",
        "ky": ""
      },
      "translations": [

      ],
      "examples": [
        {
          "ru": "класть клеймо",
          "ky": [
            "тамга салуу"
          ]
        }
      ]
    },
    {
      "description": {
        "ru": "",
        "ky": ""
      },
      "translations": [

      ],
      "examples": [
        {
          "ru": "класть все силы на работу",
          "ky": [
            "бардык күчун ишке сарп кылуу"
          ]
        }
      ]
    }
  ]
}
```

INSTRUCTIONS:
When dictionary entries are provided, use the SCHEMA DEFINITION to structure the output. Don't forget to follow the format as shown in the EXAMPLES.
REMEMBER: "translations" array should contain only Kyrgyz words or phrases. DO NOT ADD RUSSIAN words and phrases into "translations"! DO NOT INCLUDE USAGE EXAMPLE AS A TRANSLATION!
Output should only contain a JSON structure without any accompanying texts or comments. Wait until an article is given to proceed."""


In [ ]:
SYSTEM_PROMPT_23_EXAMPLES = """You are an assistant tasked with formatting old Russian-Kyrgyz dictionary entries into a computer-readable format.

In the following messages, you will be given partially formatted dictionary entries, which you need to convert into JSON format using the SCHEMA DEFINITION provided below.

SCHEMA DEFINITION:
```
{
  "type": "object",
  "properties": {
    "ru": {
      "type": "string"
    },
    "meta": {
      "type": "string"
    },
    "ky": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "description": {
            "type": "object",
            "properties": {
              "ky": {
                "type": "string"
              },
              "ru": {
                "type": "string"
              }
            },
            "required": ["ky", "ru"]
          },
          "translations": {
            "type": "array",
            "items": {
              "type": "string"
            }
          },
          "examples": {
            "type": "array",
            "items": {
              "type": "object",
              "properties": {
                "ru": {
                  "type": "string"
                },
                "ky": {
                  "type": "array",
                  "items": {
                    "type": "string"
                  }
                }
              },
              "required": ["ru", "ky"]
            }
          }
        },
        "required": ["description", "translations", "examples"]
      }
    },
    "ref": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "word": {
            "type": "string"
          },
          "description": {
            "type": "object",
            "properties": {
              "ky": {
                "type": "string"
              },
              "ru": {
                "type": "string"
              }
            },
            "required": ["ky", "ru"]
          }
        },
        "required": ["word", "description"]
      }
    }
  },
  "required": ["ru", "meta", "ky"]
}
```

DETAILS ON FIELDS:
* "ru" string field: This is the dictionary entry's key, representing the word or phrase in Russian. It acts as the main reference for the entry.
* "meta" string field: This field contains metadata about the word, providing details such as part of speech, grammatical gender, and usage notes. These details help understand the grammatical and contextual aspects of the word in Russian. For example: "ср.", "сов.", "несов.", "женск. р.", "сов. разг.", "ж. разг.", "­ая, -ое", "м.", "сов. чего, груб.", "сов. что, чего, разг.", "несов.", "нареч.", "сов. что, чего", "ср.", "несов.", "сов. что чем, офиц.", "несов.", "сов. кого", "м. лит." … etc. In most cases, this information follows the dictionary's key.
* "ky" array: Contains multiple objects each comprising fields for "description", "translations", and "examples" related to the word's usage in Kyrgyz.
    * "translations" array: Lists the translations of the Russian word into Kyrgyz. Each translation is provided as a separate string. This section should include only the translations without any additional comments or explanatory texts. REMEMBER! PUT ONLY KYRGYZ TEXTS.
    * "description" object:
        * "ky": This subfield should contain explanations, comments, or additional information about the word exclusively in Kyrgyz. It should not include translations but may contain context, usage examples, or cultural notes. REMEMBER! PUT HERE ONLY KYRGYZ TEXTS, BUT DO NOT PUT TRANSLATIONS HERE.
        * "ru": This subfield should include explanations, comments, or additional information exclusively in Russian. Like the Kyrgyz description, this should provide context, usage examples, or cultural notes relevant to the Russian language. Can only include text in Russian. REMEMBER! PUT HERE ONLY RUSSIAN TEXTS. Examples that usually go into this field: "полит.", "перен.", "кого", "что", "церк.", "театр." etc.
    * "examples" array: Contains objects that provide example sentences or phrases showcasing the use of the word and corresponding translations:
        * "ru" — Includes an example in Russian.
        * "ky" — Includes translations of the Russian example. In this array multiple examples can be listed to show different usages or nuances.
* "ref" object — Used when referring to another dictionary entry, linking related words or entries to provide additional context. This can help users understand connections between words and see related vocabulary.

EXAMPLES:
———
Entry:
обновление ср.\n1. жаңыруу, жаңыртуу, жаңылоо, жаңылануу;\nобновление методов работы иштин методдорун жаңылоо;\n2. (починка) оңдоо, түзөтүү;\n3. перен. (пополнение) жаңыртуу, калыбына келтирилүү;\nобновление знаний билимин жаңыртуу, билимин калыбына келтирүү.

Comments: Look at the "translations" fields below. Multiple Kyrgyz translations are listed in each of them there.

Computer-readable format:
```
{
  "ru": "обновление",
  "meta":  "ср.",
  "ky": [
    {
      "description": {
        "ky": "",
        "ru": ""
      },
      "translations": [
        "жаңыруу",
        "жаңыртуу",
        "жаңылоо",
        "жаңылануу"
      ],
      "examples": [
        {
          "ru": "обновление методов работы",
          "ky": [
            "иштин методдорун жаңылоо"
          ]
        }
      ]
    },
    {
      "description": {
        "ky": "",
        "ru": "починка"
      },
      "translations": [
        "оңдоо",
        "түзөтүү"
      ],
      "examples": [

      ]
    },
    {
      "description": {
        "ky": "",
        "ru": "перен. (пополнение)"
      },
      "translations": [
        "жаңыртуу",
        "калыбына келтирилүү"
      ],
      "examples": [
        {
          "ru": "обновление знаний",
          "ky": [
            "билимин жаңыртуу",
            "билимин калыбына келтирүү"
          ]
        }
      ]
    }
  ]
}
```
———
Entry:
анархизм м.\nанархизм (1. полит. майда буржуазиялык реакциялык агым; бул агым мамлекттин кандайынын болсо да, ошонун ичинде пролетариат диктатурасынын да зарылдыгын, пролетардык партиянып уюшулган саясий күрөшүнүн жана жетекчилик ролунун зарылдыгын танат; 2. перен. ээнбаштык, авторитетти тоотпоочулук; тартипке, дисциплинага моюн сунбоочулук).

Comments: Look at the "translations" field below. Since the word has the same translation as the original, Kyrgyz version looks the same.

Computer-readable format:
```
{
  "ru": "анархизм",
  "meta": "м.",
  "ky": [
    {
      "description": {
        "ky": "майда буржуазиялык реакциялык агым; бул агым мамлекттин кандайынын болсо да, ошонун ичинде пролетариат диктатурасынын да зарылдыгын, пролетардык партиянын уюшулган саясий күрөшүнүн жана жетекчилик ролунун зарылдыгын танат",
        "ru": "полит."
      },
      "translations": [
        "анархизм"
      ],
      "examples": []
    },
    {
      "description": {
        "ky": "ээнбаштык, авторитетти тоотпоочулук; тартипке, дисциплинага моюн сунбоочулук",
        "ru": "перен."
      },
      "translations": [
        "анархизм"
      ],
      "examples": []
    }
  ]
}
```
———
Entry:
обновляться несов.\n1. см. обновиться;\n2. страд. к обновлять.

Comment: In the output below, no Kyrgyz translation is provided, but references to other dictionary entries are included.

Computer-readable format:
```
{
  "ru": "обновляться",
  "meta":  "несов.",
  "ref": [
    {
      "word": "обновиться",
      "description": {
        "ky": "",
        "ru": "см."
      }
    },
    {
      "word": "обновлять",
      "description": {
        "ky": "",
        "ru": "страд. к"
      }
    }
  ]
}
```
———
Entry:
чужеземка женск. р. к чужеземец.

Comment: In the example below there's no Kyrgyz translation, but it has a referenc to another dictionary entry.

Computer-readable format:
```
{
  "ru": "чужеземка",
  "meta":  "женск. р.",
  "ref": [
    {
      "word": "чужеземец",
      "description": {
        "ky": "",
        "ru": "к"
      }
    }
  ]
}
```
———
Entry:
обноситься сов. разг.\n1. (износить одежду) кийими жыртылып бүтүү;\n2. (стать удобным) кийилип жүрүп ык алуу, калып алуу;\nваленки обносились кийиз өтүк бутка калып алып калды;\n3. (обветшать от носки) эскирүү, тамтыгы чыгуу;\nплатье обносилось көйнөк эскирди.

Comments: Look at the "translations" field below. Multiple Kyrgyz translations are listed in each of them there.

Computer-readable format:
```
{
  "ru": "обноситься",
  "meta": "сов. разг.",
  "ky": [
    {
      "description": {
        "ky": "",
        "ru": "износить одежду"
      },
      "translations": [
        "кийими жыртылып бүтүү"
      ],
      "examples": [

      ]
    },
    {
      "description": {
        "ky": "",
        "ru": "стать удобным"
      },
      "translations": [
        "кийилип жүрүп ык алуу",
        "калып алуу"
      ],
      "examples": [
        {
          "ru": "валенки обносились",
          "ky": [
            "кийиз өтүк бутка калып алып калды"
          ]
        }
      ]
    },
    {
      "description": {
        "ky": "",
        "ru": "обветшать от носки"
      },
      "translations": [
        "эскирүү",
        "тамтыгы чыгуу"
      ],
      "examples": [
        {
          "ru": "платье обносилось",
          "ky": [
            "көйнөк эскирди"
          ]
        }
      ]
    }
  ]
}
```
———
Entry:
обновка ж. разг.\nжаңы алынган буюм (мис. жаңы кийим).

Comments: Look the "translations" field below. A Kyrgyz translation listed there.

Computer-readable format:
```
{
  "ru": "обновка",
  "meta": "ж. разг.",
  "ky": [
    {
      "description": {
        "ky": "мис. жаңы кийим",
        "ru": ""
      },
      "translations": [
        "жаңы алынган буюм"
      ],
      "examples": [

      ]
    }
  ]
}
```
———
Entry:
православный, ­ая, -ое\n1. православие-ге т.;\nправославная вера православие дини;\n2. в знач. сущ. м., ж. православие дининдеги киши.

Comment: In the example below, "translations" is empty in one instance because the translation is not provided separately but is given through the "examples".

Computer-readable format:
```
{
  "ru": "православный",
  "meta": "­ая, -ое",
  "ky": [
    {
      "description": {
        "ky": "православие-ге т.",
        "ru": ""
      },
      "translations": [

      ],
      "examples": [
        {
          "ru": "православная вера",
          "ky": [
            "православие дини"
          ]
        }
      ]
    },
    {
      "description": {
        "ky": "",
        "ru": "в знач. сущ. м., ж."
      },
      "translations": [
        "православие дининдеги киши"
      ],
      "examples": [

      ]
    }
  ]
}
```
———
Entry:
автопарк м.\nавтопарк (автомобилдер туруучу, ремонттолуучу жай).

Comments: Look at the "translations" field below. Since the word has the same translation as the original, Kyrgyz version looks the same.

Computer-readable format:
```
{
  "ru": "автопарк",
  "meta": "м.",
  "ky": [
    {
      "description": {
        "ru": "",
        "ky": "автомобилдер туруучу, ремонттолуучу жай"
      },
      "translations": [
        "автопарк"
      ],
      "examples": [

      ]
    }
  ]
}
```
———
Entry:
нажраться сов. чего, груб.\nтоюу (алпылдап сугунуп, алпылдап жеп-ичип).

Comments: Look at the "translations" field below. A Kyrgyz translation is listed there.

Computer-readable format:
```
{
  "ru": "нажраться",
  "meta": "сов. чего, груб.",
  "ky": [
    {
      "description": {
        "ru": "",
        "ky": "алпылдап сугунуп, алпылдап жеп-ичип"
      },
      "translations": [
        "тоюу"
      ],
      "examples": [

      ]
    }
  ]
}
```
———
Entry:
накидать сов. что, чего, разг.\nчачып таштоо, таштай берүү, ыргыта берүү;\nсм. набросать I.

Comments: Look at the "translations" field below. Multiple Kyrgyz translations are listed there.

Computer-readable format:
```
{
  "ru": "накидать",
  "meta": "сов. что, чего, разг.",
  "ref": [
    {
      "word": "набросать",
      "description": {
        "ru": "см.",
        "ky": ""
      },

    }
  ],
  "ky": [
    {
      "description": {
        "ru": "",
        "ky": ""
      },
      "translations": [
        "чачып таштоо",
        "таштай берүү",
        "ыргыта берүү"
      ],
      "examples": [

      ]
    }
  ]
}
```
———
Entry:
налегать несов.\n1. см. налечь;\n2. (лежать сверху) кабатталып жатуу;\nгорные породы налегают одна на другую тоо тектери биринин үстүндө бири кабатталып жатат.

Comments: Look at the "translations" field below. A Kyrgyz translation is listed there.

Computer-readable format:
```
{
  "ru": "налегать",
  "meta": "несов.",
  "ref": [
    {
      "word": "налечь",
      "description": {
        "ru": "см.",
        "ky": ""
      }
    }
  ],
  "ky": [
    {
      "description": {
        "ru": "лежать сверху",
        "ky": ""
      },
      "translations": [
        "кабатталып жатуу"
      ],
      "examples": [
        {
          "ru": "горные породы налегают одна на другую",
          "ky": [
            "тоо тектери биринин үстүндө бири кабатталып жатат"
          ]
        }
      ]
    }
  ]
}
```
———
Entry:
налево нареч.\nсолго, сол тарапка, сол жакка (в левую сторону); сол тарапта, сол жакта (на левой стороне);\nпрохожий свернул налево жолоочу сол жакка бурулуп кетти;\nналево от дома - лес үйдүн сол жагында токой бар.

Comments: Look at the "translations" fields below. Multiple Kyrgyz translations are listed in each of them.

Computer-readable format:
```
{
  "ru": "налево",
  "meta": "нареч.",
  "ky": [
    {
      "description": {
        "ru": "в левую сторону",
        "ky": ""
      },
      "translations": [
        "солго",
        "сол тарапка",
        "сол жакка"
      ],
      "examples": [
        {
          "ru": "прохожий свернул налево",
          "ky": [
            "жолоочу сол жакка бурулуп кетти"
          ]
        }
      ]
    },
    {
      "description": {
        "ru": "на левой стороне",
        "ky": ""
      },
      "translations": [
        "сол тарапта",
        "сол жакта"
      ],
      "examples": [
        {
          "ru": "налево от дома - лес",
          "ky": [
            "үйдүн сол жагында токой бар"
          ]
        }
      ]
    }
  ]
}
```
———
Entry:
намариновать сов. что, чего\nмаринаддоо (түрдүү жемишти ачык сууга салуу, ачытуу, кычкылдантуу).

Comments: In the example below, the explanation "(түрдүү жемишти ачык сууга салуу, ачытуу, кычкылдантуу)" has been included in the "ky" section of the dictionary's "description" because it is an explanation of the translation.

Computer-readable format:
```
{
  "ru": "намариновать",
  "meta": "сов. что, чего",
  "ky": [
    {
      "description": {
        "ru": "",
        "ky": "түрдүү жемишти ачык сууга салуу, ачытуу, кычкылдантуу"
      },
      "translations": [
        "маринаддоо"
      ],
      "examples": [

      ]
    }
  ]
}
```
———
Entry:
материнство ср.\nэнелик (1. эненин балага сезими, туйгусу; 2. аялдын боюнда бар, төрөт жана бала эмизүү убагындагы абалы).

Comments: See the "translations" field? A Kyrgyz translation is listed there. Also, see the "description" field's usage example.

Computer-readable format:
```
{
  "ru": "материнство",
  "meta": "ср.",
  "ky": [
    {
      "description": {
        "ru": "",
        "ky": "эненин балага сезими, туйгусу; аялдын боюнда бар, төрөт жана бала эмизүү убагындагы абалы"
      },
      "translations": [
        "энелик"
      ],
      "examples": [

      ]
    }
  ]
}
```
———
Entry:
вертеть несов.\n1. что, чем (вращать) дөңгөлөтүү, айландыруу, тегеретүү;\nвертеть колесо дөңгөлөктү айландыруу (тегеретүү, дөңгөлөтүү);\nвертеть тростью колу менен таякты тегеретүү;\n2. кем-чем, перен. разг. чайкоо, өз билгенин кылып бирөөнү башкаруу.

Comments: In this example, the phrase "вертеть колесо дөңгөлөктү айландыруу (тегеретүү, дөңгөлөтүү)" is provided in the dictionary entry as "вертеть колесо дөңгөлөктү айландыруу (тегерүү, дөңгөлөтүү)", but the words "тегерүү" and "дөңгөлөтүү" are given as alternatives to "айландыруу", meaning they can replace it. Therefore, examples involving those words are listed separately.

Computer-readable format:
```
{
  "ru": "вертеть",
  "meta": "несов.",
  "ky": [
    {
      "description": {
        "ru": "",
        "ky": ""
      },
      "translations": [
        "дөңгөлөтүү",
        "айландыруу",
        "тегеретүү"
      ],
      "examples": [
        {
          "ru": "вертеть колесо",
          "ky": [
            "дөңгөлөктү айландыруу",
            "дөңгөлөктү тегеретүү",
            "дөңгөлөктү дөңгөлөтүү"
          ]
        },
        {
          "ru": "вертеть тростью",
          "ky": [
            "колу менен таякты тегеретүү"
          ]
        }
      ]
    },
    {
      "description": {
        "ru": "кем-чем, перен. разг.",
        "ky": ""
      },
      "translations": [
        "чайкоо",
        "өз билгенин кылып бирөөнү башкаруу"
      ],
      "examples": [

      ]
    }
  ]
}
```
———
Entry:
сопроводить сов. что чем, офиц.\nбирге (кошо) жиберүү;\nсопроводить заявление справкой арызды справка менен кошо жиберүү.

Comments: Look at the "translations" field below. Multiple Kyrgyz translation are listed there. Also, see the "description" field's usage example.

Computer-readable format:
```
{
  "ru": "сопроводить",
  "meta": "сов. что чем, офиц.",
  "ky": [
    {
      "description": {
        "ru": "",
        "ky": ""
      },
      "translations": [
        "бирге жиберүү",
        "кошо жиберүү"
      ],
      "examples": [
        {
          "ru": "сопроводить заявление справкой",
          "ky": [
            "арызды справка менен кошо жиберүү"
          ]
        }
      ]
    }
  ]
}
```
———
Entry:
класть несов.\n1. кого-что коюу;\nкласть на место ордуна коюу;\n2. кого (помещать - напр. в больницу) жаткыруу (мис. төшөккө, ооруканага);\n3. что (напр. в банк) салуу (мис. капка, банкага);\n4. что (возводить, строить) тургузуу, жасоо, салуу (мис. дубалды, мешти);\nкласть в рот толук тушүндүрүү, кулагына куюу;\nкласть яйца жумуртка салуу (канаттуулар менен курт-кумурскалардын ургаачылары жөнүндө);\nкласть начало баштоо;\nкласть основание негиздөө, түптөө;\nкласть печать печать басуу;\nкласть клеймо тамга салуу;\nкласть все силы на работу бардык күчун ишке сарп кылуу.

Comments: In this example, "translations" is empty in some instances because there the translation is given through the "examples". Also, pay attention to how "description" field is used.

Computer-readable format:
```
{
  "ru": "класть",
  "meta": "несов.",
  "ky": [
    {
      "description": {
        "ru": "",
        "ky": ""
      },
      "translations": [
        "коюу"
      ],
      "examples": [
        {
          "ru": "класть на место",
          "ky": [
            "ордуна коюу"
          ]
        }
      ]
    },
    {
      "description": {
        "ru": "кого (помещать - напр. в больницу)",
        "ky": "мис. төшөккө, ооруканага"
      },
      "translations": [
        "жаткыруу"
      ],
      "examples": [

      ]
    },
    {
      "description": {
        "ru": "что (напр. в банк)",
        "ky": "мис. капка, банкага"
      },
      "translations": [
        "салуу"
      ],
      "examples": [

      ]
    },
    {
      "description": {
        "ru": "что (возводить, строить)",
        "ky": "мис. дубалды, мешти"
      },
      "translations": [
        "тургузуу",
        "жасоо",
        "салуу"
      ],
      "examples": [

      ]
    },
    {
      "description": {
        "ru": "",
        "ky": ""
      },
      "translations": [

      ],
      "examples": [
        {
          "ru": "класть в рот",
          "ky": [
            "толук тушүндүрүү",
            "кулагына куюу"
          ]
        }
      ]
    },
    {
      "description": {
        "ru": "",
        "ky": "канаттуулар менен курт-кумурскалардын ургаачылары жөнүндө"
      },
      "translations": [

      ],
      "examples": [
        {
          "ru": "класть яйца",
          "ky": [
            "жумуртка салуу"
          ]
        }
      ]
    },
    {
      "description": {
        "ru": "",
        "ky": ""
      },
      "translations": [

      ],
      "examples": [
        {
          "ru": "класть начало",
          "ky": [
            "баштоо"
          ]
        }
      ]
    },
    {
      "description": {
        "ru": "",
        "ky": ""
      },
      "translations": [

      ],
      "examples": [
        {
          "ru": "класть основание",
          "ky": [
            "негиздөө",
            "түптөө"
          ]
        }
      ]
    },
    {
      "description": {
        "ru": "",
        "ky": ""
      },
      "translations": [

      ],
      "examples": [
        {
          "ru": "класть печать",
          "ky": [
            "печать басуу"
          ]
        }
      ]
    },
    {
      "description": {
        "ru": "",
        "ky": ""
      },
      "translations": [

      ],
      "examples": [
        {
          "ru": "класть клеймо",
          "ky": [
            "тамга салуу"
          ]
        }
      ]
    },
    {
      "description": {
        "ru": "",
        "ky": ""
      },
      "translations": [

      ],
      "examples": [
        {
          "ru": "класть все силы на работу",
          "ky": [
            "бардык күчун ишке сарп кылуу"
          ]
        }
      ]
    }
  ]
}
```
———
Entry:
антивоенный, ­ая, -ое\nсогушка каршы;\nантивоенная демонстрация согушка каршы демонстрация.

Comments: Pay attention to the "description", "translations" and "examples" and how they are used.

Computer-readable format:
```
{
  "ru": "антивоенный",
  "meta": "­ая, -ое",
  "ky": [
    {
      "description": {
        "ky": "",
        "ru": ""
      },
      "translations": [
        "согушка каршы"
      ],
      "examples": [
        {
          "ru": "антивоенная демонстрация",
          "ky": [
            "согушка каршы демонстрация"
          ]
        }
      ]
    }
  ]
}
```
———
Entry:
анонимный, анонимдүү, тоголок, туюк (автордун аты жазылбаган);\nанонимное письмо анонимдүү кат, тоголок кат.

Comments: Look at the "translations" field below. Multiple Kyrgyz translation are listed there. Also, see the "description" field's usage example.

Computer-readable format:
```
{
  "ru": "анонимный",
  "meta": "",
  "ky": [
    {
      "description": {
        "ky": "автордун аты жазылбаган",
        "ru": ""
      },
      "translations": [
        "анонимдүү",
        "тоголок",
        "туюк"
      ],
      "examples": [
        {
          "ru": "анонимное письмо",
          "ky": [
            "анонимдүү кат",
            "тоголок кат"
          ]
        }
      ]
    }
  ]
}
```
———
Entry:
безграничность ж.\nчексиздик, учу-кыйыры жоктук.

Comments: Look at the "translations" field below. Multiple Kyrgyz translation are listed there.

Computer-readable format:
```
{
  "ru": "безграничность",
  "meta": "ж.",
  "ky": [
    {
      "description": {
        "ky": "",
        "ru": ""
      },
      "translations": [
        "чексиздик",
        "учу-кыйыры жоктук"
      ],
      "examples": [

      ]
    }
  ]
}
```
———
Entry:
безбожно нареч. разг.\nыксыз, кудайды карабай, чактабай;\nбезбожно врать ыксыз калп айтуу.

Comments: Look at the "translations" field below. Multiple Kyrgyz translation are listed there. Also, see the "description" field's usage example.

Computer-readable format:
```
{
  "ru": "безбожно",
  "meta": "нареч. разг.",
  "ky": [
    {
      "description": {
        "ky": "",
        "ru": ""
      },
      "translations": [
        "ыксыз",
        "кудайды карабай",
        "чактабай"
      ],
      "examples": [
        {
          "ru": "безбожно врать",
          "ky": [
            "ыксыз калп айтуу"
          ]
        }
      ]
    }
  ]
}
```
———
Entry:
благодарный ­ая, -ое\n1. ыраазы;\nя вам очень благодарен мен сизге абдан ыраазымын;\n2. ийгиликтүү, убайлуу;\nблагодарный труд ийгиликтүү эмгек.

Comments: Look at the "translations" field below. Multiple Kyrgyz translation are listed in each of them. Also, see the "description" field's usage example.

Computer-readable format:
```
{
  "ru": "благодарный",
  "meta": "­ая, -ое",
  "ky": [
    {
      "description": {
        "ky": "",
        "ru": ""
      },
      "translations": [
        "ыраазы"
      ],
      "examples": [
        {
          "ru": "я вам очень благодарен",
          "ky": [
            "мен сизге абдан ыраазымын"
          ]
        }
      ]
    },
    {
      "description": {
        "ky": "",
        "ru": ""
      },
      "translations": [
        "ийгиликтүү",
        "убайлуу"
      ],
      "examples": [
        {
          "ru": "благодарный труд",
          "ky": [
            "ийгиликтүү эмгек"
          ]
        }
      ]
    }
  ]
}
```
———
Entry:
бок м.\nкаптал, жан; жак (сторона);\nу меня колет в боку менин капталым сайгылашып ооруп турат;\nвьюк свесился на левый бок жүк сол жакка ооп калды;\nбок о бок жанаша, катар;\nлежать на боку разг. (бездельничать) бекер жүрүү;\nпод боком разг. эң жакын, жанында;\nвзять за бока разг. асылуу, кекиртектен алуу;\nпереваливатъся с боку на бок (при ходьбе) оонап басуу, чайкалып басуу, өрдөкчө басуу;\nсомнения по боку! разг. ыргылжыңды кой!

Comments: In this example, "translations" is empty in some instances because there the translation is given through the "examples". Also, pay attention to how "description" field is used.

Computer-readable format:
```
{
  "ru": "бок",
  "meta": "м.",
  "ky": [
    {
      "description": {
        "ky": "",
        "ru": "сторона"
      },
      "translations": [
        "каптал",
        "жан",
        "жак"
      ],
      "examples": [
        {
          "ru": "у меня колет в боку",
          "ky": [
            "менин капталым сайгылашып ооруп турат"
          ]
        },
        {
          "ru": "вьюк свесился на левый бок",
          "ky": [
            "жүк сол жакка ооп калды"
          ]
        }
      ]
    },
    {
      "description": {
        "ky": "",
        "ru": ""
      },
      "translations": [

      ],
      "examples": [
        {
          "ru": "бок о бок",
          "ky": [
            "жанаша",
            "катар"
          ],

        }
      ]
    },
    {
      "description": {
        "ky": "",
        "ru": "разг. (бездельничать)"
      },
      "translations": [

      ],
      "examples": [
        {
          "ru": "лежать на боку",
          "ky": [
            "бекер жүрүү"
          ]
        }
      ]
    },
    {
      "description": {
        "ky": "",
        "ru": "разг."
      },
      "translations": [

      ],
      "examples": [
        {
          "ru": "под боком",
          "ky": [
            "эң жакын",
            "жанында"
          ]
        }
      ]
    },
    {
      "description": {
        "ky": "",
        "ru": "разг."
      },
      "translations": [

      ],
      "examples": [
        {
          "ru": "взять за бока",
          "ky": [
            "асылуу",
            "кекиртектен алуу"
          ]
        }
      ]
    },
    {
      "description": {
        "ky": "",
        "ru": "при ходьбе"
      },
      "translations": [

      ],
      "examples": [
        {
          "ru": "переваливатъся с боку на бок",
          "ky": [
            "оонап басуу",
            "чайкалып басуу",
            "өрдөкчө басуу"
          ]
        }
      ]
    },
    {
      "description": {
        "ky": "",
        "ru": "разг."
      },
      "translations": [

      ],
      "examples": [
        {
          "ru": "сомнения по боку!",
          "ky": [
            "ыргылжыңды кой!"
          ]
        }
      ]
    }
  ]
}
```

INSTRUCTIONS:
When dictionary entries are provided, use the SCHEMA DEFINITION to structure the output. Don't forget to follow the format as shown in the EXAMPLES.
REMEMBER: "translations" array should contain only Kyrgyz words or phrases. DO NOT ADD RUSSIAN words and phrases into "translations"! DO NOT INCLUDE USAGE EXAMPLE AS A TRANSLATION!
Output should only contain a JSON structure without any accompanying texts or comments. Wait until an article is given to proceed."""


In [ ]:
import time
import json
from google.colab import userdata
from mistralai import Mistral

SLEEP_TIME = 0.5

# Initialize the Mistral client
client = Mistral(api_key=userdata.get('MISTRAL_API_KEY'))

def get_inference_with_few_shot_examples_from_mistral(output_file, system_prompt, model_name='open-mistral-nemo'):
    outputs = []
    processed_task_ids = set()

    # Load previously processed task IDs
    try:
        with open(output_file, 'r', encoding='utf-8') as f:
            for line in f:
                data = json.loads(line)
                processed_task_ids.add(data['task-id'])
    except FileNotFoundError:
        pass

    # Helper function to save results
    def save_results():
        with open(output_file, 'w', encoding='utf-8') as f:
            for item in outputs:
                f.write(json.dumps(item, ensure_ascii=False) + '\n')
        print(f"Progress saved to {output_file}")

    # Process the input corpus
    with open('gold_corpus_structured_yudakhin.jsonl', 'r') as f:
        for line in f:
            try:
                data = json.loads(line)
                task_id = data.get('task-id')

                # Skip tasks that have already been processed
                if task_id in processed_task_ids:
                    continue

                dict_entry = data.get('text')

                # Call the Mistral API
                try:
                    response = client.chat.complete(
                        model=model_name,
                        messages=[
                            {"role": "system", "content": system_prompt},
                            {"role": "user", "content": dict_entry}
                        ],
                        temperature=0,
                        max_tokens=4096,
                        top_p=1
                    )
                except Exception as e:
                    print(f'Exception processing word "{dict_entry}": {e}')
                    continue

                # Append the result
                outputs.append({
                    'task-id': task_id,
                    'text': dict_entry,
                    'json': response.choices[0].message.content
                })

                # Save results periodically
                if len(outputs) % 10 == 0:
                    save_results()

            except Exception as e:
                print(f"General error for task-id {task_id}: {e}")
                time.sleep(60)  # Sleep for 1 minute and continue

    # Final save after finishing
    save_results()


In [ ]:
output_file = 'untrained_mistral_nemo_outputs.jsonl'
get_inference_with_few_shot_examples_from_mistral(output_file, zero_shot_system_message, model_name='open-mistral-nemo')

Progress saved to untrained_mistral_nemo_outputs.jsonl
Progress saved to untrained_mistral_nemo_outputs.jsonl
Progress saved to untrained_mistral_nemo_outputs.jsonl
Progress saved to untrained_mistral_nemo_outputs.jsonl
Progress saved to untrained_mistral_nemo_outputs.jsonl
Progress saved to untrained_mistral_nemo_outputs.jsonl
Progress saved to untrained_mistral_nemo_outputs.jsonl
Progress saved to untrained_mistral_nemo_outputs.jsonl
Progress saved to untrained_mistral_nemo_outputs.jsonl
Progress saved to untrained_mistral_nemo_outputs.jsonl
Progress saved to untrained_mistral_nemo_outputs.jsonl


In [ ]:
output_file = 'untrained_mistral_nemo_outputs_6_examples.jsonl'
get_inference_with_few_shot_examples_from_mistral(output_file, SYSTEM_PROMPT_6_EXAMPLES, model_name='open-mistral-nemo')

Progress saved to untrained_mistral_nemo_outputs_6_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_6_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_6_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_6_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_6_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_6_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_6_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_6_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_6_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_6_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_6_examples.jsonl


In [ ]:
output_file = 'untrained_mistral_nemo_outputs_12_examples.jsonl'
get_inference_with_few_shot_examples_from_mistral(output_file, SYSTEM_PROMPT_12_EXAMPLES, model_name='open-mistral-nemo')

Progress saved to untrained_mistral_nemo_outputs_12_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_12_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_12_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_12_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_12_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_12_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_12_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_12_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_12_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_12_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_12_examples.jsonl


In [ ]:
output_file = 'untrained_mistral_nemo_outputs_17_examples.jsonl'
get_inference_with_few_shot_examples_from_mistral(output_file, SYSTEM_PROMPT_17_EXAMPLES, model_name='open-mistral-nemo')

Progress saved to untrained_mistral_nemo_outputs_17_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_17_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_17_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_17_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_17_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_17_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_17_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_17_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_17_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_17_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_17_examples.jsonl


In [ ]:
output_file = 'untrained_mistral_nemo_outputs_23_examples.jsonl'
get_inference_with_few_shot_examples_from_mistral(output_file, SYSTEM_PROMPT_23_EXAMPLES, model_name='open-mistral-nemo')

Progress saved to untrained_mistral_nemo_outputs_23_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_23_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_23_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_23_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_23_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_23_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_23_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_23_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_23_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_23_examples.jsonl
Progress saved to untrained_mistral_nemo_outputs_23_examples.jsonl


# Get output from the fine-tuned mistral-nemo

In [ ]:
json_schema = {
  "type": "object",
  "properties": {
    "ru": {
      "type": "string"
    },
    "meta": {
      "type": "string"
    },
    "ky": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "description": {
            "type": "object",
            "properties": {
              "ky": {
                "type": "string"
              },
              "ru": {
                "type": "string"
              }
            },
            "required": ["ky", "ru"]
          },
          "translations": {
            "type": "array",
            "items": {
              "type": "string"
            }
          },
          "examples": {
            "type": "array",
            "items": {
              "type": "object",
              "properties": {
                "ru": {
                  "type": "string"
                },
                "ky": {
                  "type": "array",
                  "items": {
                    "type": "string"
                  }
                }
              },
              "required": ["ru", "ky"]
            }
          }
        },
        "required": ["description", "translations", "examples"]
      }
    },
    "ref": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "word": {
            "type": "string"
          },
          "description": {
            "type": "object",
            "properties": {
              "ky": {
                "type": "string"
              },
              "ru": {
                "type": "string"
              }
            },
            "required": ["ky", "ru"]
          }
        },
        "required": ["word", "description"]
      }
    }
  },
  "required": ["ru", "meta", "ky"]
}

system_message = f"""You are an expert assistant tasked with converting Russian-Kyrgyz dictionary entries into a structured JSON format. Your output should strictly adhere to the given schema, ensuring the data is well-organized and accurate. Follow the field guidelines below:

Field Details:
- "ru": A string field representing the dictionary entry’s key, which is the word or phrase in Russian. This field acts as the main reference for the entry.
- "meta": A string field containing metadata about the word, such as part of speech, grammatical gender, and usage notes. Examples of metadata include "ср.", "сов.", "несов.", "нареч." etc.
- "ky": An array containing multiple objects. Each object includes the following fields:
    - "translations": An array listing the Kyrgyz translations of the Russian word. Each translation is provided as a separate string, and this section must contain only Kyrgyz texts, without any additional explanations or comments.
    - "description": An object with the following subfields:
        - "ky": Contains explanations or additional information about the word, written exclusively in Kyrgyz. Do not include translations here, but you may provide context, usage notes, or cultural explanations.
        - "ru": Contains explanations or additional information written exclusively in Russian, providing context, usage, or cultural notes relevant to the word. Examples could include "полит.", "перен.", "церк.", etc. This section must only include Russian text.
    - "examples": An array of objects that provide example sentences to illustrate the word’s usage:
        - "ru": An example sentence in Russian.
        - "ky": The translation of the example sentence in Kyrgyz. You can list multiple translations to showcase different nuances.
- "ref": An object for linking related dictionary entries, helping users understand connections between words and offering additional context.

Use the schema provided below to structure your output accordingly:

{json_schema}
"""

In [ ]:
output_file = 'finedtuned_mistral_nemo_outputs.jsonl'
model_name = 'ft:open-mistral-nemo:e314d8ea:20240921:8f47ee95'
get_inference_with_few_shot_examples_from_mistral(output_file, system_message, model_name)

Progress saved to finedtuned_mistral_nemo_outputs.jsonl
Progress saved to finedtuned_mistral_nemo_outputs.jsonl
Progress saved to finedtuned_mistral_nemo_outputs.jsonl
Progress saved to finedtuned_mistral_nemo_outputs.jsonl
Progress saved to finedtuned_mistral_nemo_outputs.jsonl
Progress saved to finedtuned_mistral_nemo_outputs.jsonl
Progress saved to finedtuned_mistral_nemo_outputs.jsonl
Progress saved to finedtuned_mistral_nemo_outputs.jsonl
Progress saved to finedtuned_mistral_nemo_outputs.jsonl
Progress saved to finedtuned_mistral_nemo_outputs.jsonl
Progress saved to finedtuned_mistral_nemo_outputs.jsonl
